#Сравнение энергий, посчитанных с помощью полученных формул обменно-корреляционных функционалов (известных и нейросетевого), посчитанных с помощью нейросетевого функционала напрямую, а также референсного (из программного пакета pycsf)

Источник исходного кода - https://github.com/ml-electron-project/NNfunctional/tree/master

Установим необходимые зависимости

In [1]:
%env PYTHONPATH = # /env/python

env: PYTHONPATH=# /env/python


In [2]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -y -p /usr/local
!conda update conda -y

--2023-07-28 06:20:14--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76120962 (73M) [application/x-sh]
Saving to: ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’

Miniconda3-py38_4.1 100%[===================>]  72.59M   185MB/s    in 0.4s    

2023-07-28 06:20:14 (185 MB/s) - ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’ saved [76120962/76120962]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py38h06a4308_2
    - cffi==1.15.0=py38hd667e15_1
    - ch

In [3]:
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

In [4]:
!conda create -y -n myenv python=3.6

Solving environment: | / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py36h06a4308_0         139 KB
    openssl-1.1.1u             |       h7f8727e_0         3.7 MB
    pip-21.2.2                 |   py36h06a4308_0         1.8 MB
    python-3.6.13              |       h12debd9_1        32.5 MB
    setuptools-58.0.4          |   py36h06a4308_0         788 KB
    ------------------------------

In [5]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install pyscf==1.6.2 torch==1.1.0

     |████████████████████████████████| 21.5 MB 339 kB/s 
     |████████████████████████████████| 676.9 MB 4.0 kB/s 
     |████████████████████████████████| 14.8 MB 46.0 MB/s 
     |████████████████████████████████| 4.0 MB 49.6 MB/s 
     |████████████████████████████████| 25.9 MB 111.7 MB/s 


In [6]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip show pyscf

Name: pyscf
Version: 1.6.2
Summary: PySCF: Python-based Simulations of Chemistry Framework
Home-page: http://www.pyscf.org
Author: Qiming Sun
Author-email: osirpt.sun@gmail.com
License: Apache License 2.0
Location: /usr/local/envs/myenv/lib/python3.6/site-packages
Requires: scipy, numpy, h5py
Required-by: 


In [7]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
mv /content/numint.py /usr/local/envs/myenv/lib/python3.6/site-packages/pyscf/dft/numint.py

##Приступим непосредственно к расчету энергий. Нас интересует значение в последней строчке вывода - "converged SCF energy = ..."

##Энергия, полученная с использованием напрямую посчитанного нейросетевого обменно-корреляционного потенциала

In [8]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 '/content/LSDA_NN.py'

#INFO: **** input file is /content/LSDA_NN.py ****
# -*- coding: utf-8 -*-
#!/usr/bin/env python
import numpy as np
from pyscf import gto, dft
import math
import torch 
from torch.autograd import Variable 
import torch.nn as nn 
import torch.nn.functional as F 


# definition of target molecule #
mol = gto.Mole()
mol.verbose = 4

mol.atom="""8            .000000     .000000     .119262
1            .000000     .763239    -.477047
1            .000000    -.763239    -.477047"""
mol.charge=0
mol.spin  =0
mol.basis = "6-31G"
mol.build()

# definition of NN structure #
hidden=100
print("hidden nodes= "+str(hidden))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, 1)


    def forward(self, x):
        t=torch.empty((x.shape[0],2))
        unif=(x[:,1]+x[:,0]+1e-7)**(1.0/3)
        t[:,0]=unif

#Энергия, полученная с использованием обменно-корреляционного функционала lda_x + lda_c_chachiyo, посчитанного с помощью выражений Meta NN символьной регрессии

In [9]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 '/content/LSDA_symbolic_DL_regression (lda_x + lda_c_chachiyo).py'

#INFO: **** input file is /content/LSDA_symbolic_DL_regression (lda_x + lda_c_chachiyo).py ****
# -*- coding: utf-8 -*-
#!/usr/bin/env python
import numpy as np
from pyscf import gto, dft
import math
import torch 
from torch.autograd import Variable 
import torch.nn as nn 
import torch.nn.functional as F 


# definition of target molecule #
mol = gto.Mole()
mol.verbose = 4

mol.atom="""8            .000000     .000000     .119262
1            .000000     .763239    -.477047
1            .000000    -.763239    -.477047"""
mol.charge=0
mol.spin  =0
mol.basis = "6-31G"
mol.build()

# definition of NN structure #
hidden=100
print("hidden nodes= "+str(hidden))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, 1)


    def forward(self, x):
        t=torch.empty((x.shape[0],2))
        unif=(x[:

#Энергия, полученная с использованием обменно-корреляционного функционала lda_x + lda_c_chachiyo, посчитанного с помощью выражений GA символьной регрессии

In [10]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 '/content/LSDA_symbolic_GA_regression (lda_x + lda_c_chachiyo).py'

#INFO: **** input file is /content/LSDA_symbolic_GA_regression (lda_x + lda_c_chachiyo).py ****
# -*- coding: utf-8 -*-
#!/usr/bin/env python
import numpy as np
from pyscf import gto, dft
import math
import torch 
from torch.autograd import Variable 
import torch.nn as nn 
import torch.nn.functional as F 


# definition of target molecule #
mol = gto.Mole()
mol.verbose = 4

mol.atom="""8            .000000     .000000     .119262
1            .000000     .763239    -.477047
1            .000000    -.763239    -.477047"""
mol.charge=0
mol.spin  =0
mol.basis = "6-31G"
mol.build()

# definition of NN structure #
hidden=100
print("hidden nodes= "+str(hidden))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, 1)


    def forward(self, x):
        t=torch.empty((x.shape[0],2))
        unif=(x[:

#Энергия, полученная с использованием нейросетевого обменно-корреляционного функционала, посчитанного с помощью выражений Meta NN символьной регрессии

In [11]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 '/content/LSDA_symbolic_DL_regression (NN-based).py'

#INFO: **** input file is /content/LSDA_symbolic_DL_regression (NN-based).py ****
# -*- coding: utf-8 -*-
#!/usr/bin/env python
import numpy as np
from pyscf import gto, dft
import math
import torch 
from torch.autograd import Variable 
import torch.nn as nn 
import torch.nn.functional as F 


# definition of target molecule #
mol = gto.Mole()
mol.verbose = 4

mol.atom="""8            .000000     .000000     .119262
1            .000000     .763239    -.477047
1            .000000    -.763239    -.477047"""
mol.charge=0
mol.spin  =0
mol.basis = "6-31G"
mol.build()

# definition of NN structure #
hidden=100
print("hidden nodes= "+str(hidden))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, 1)


    def forward(self, x):
        t=torch.empty((x.shape[0],2))
        unif=(x[:,1]+x[:,0]+1e-

#Энергия, полученная с использованием нейросетевого обменно-корреляционного функционала, посчитанного с помощью выражений GA символьной регрессии

In [12]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python3 '/content/LSDA_symbolic_GA_regression (NN-based).py'

#INFO: **** input file is /content/LSDA_symbolic_GA_regression (NN-based).py ****
# -*- coding: utf-8 -*-
#!/usr/bin/env python
import numpy as np
from pyscf import gto, dft
import math
import torch 
from torch.autograd import Variable 
import torch.nn as nn 
import torch.nn.functional as F 


# definition of target molecule #
mol = gto.Mole()
mol.verbose = 4

mol.atom="""8            .000000     .000000     .119262
1            .000000     .763239    -.477047
1            .000000    -.763239    -.477047"""
mol.charge=0
mol.spin  =0
mol.basis = "6-31G"
mol.build()

# definition of NN structure #
hidden=100
print("hidden nodes= "+str(hidden))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, 1)


    def forward(self, x):
        t=torch.empty((x.shape[0],2))
        unif=(x[:,1]+x[:,0]+1e-